In [1]:
import os
import pandas as pd
import shutil

import cv2
import torch
from PIL import Image

import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_train=pd.read_csv("dataset/train.csv")
names=list(df_train["name"].unique())
mapping_dic_name_class={}
mapping_dic_class_name={}
for name in names:
    df_name=df_train[df_train["name"]==name]
    classes=list(df_name["class"].unique())
    print(name,classes)
    mapping_dic_name_class[name]=classes[0]
    mapping_dic_class_name[classes[0]]=name

GARBAGE [3.0]
BAD_BILLBOARD [7.0]
SAND_ON_ROAD [8.0]
GRAFFITI [0.0]
POTHOLES [2.0]
CLUTTER_SIDEWALK [9.0]
CONSTRUCTION_ROAD [4.0]
BROKEN_SIGNAGE [5.0]
UNKEPT_FACADE [10.0]
FADED_SIGNAGE [1.0]
BAD_STREETLIGHT [6.0]


In [3]:
# lets copy all test files to a folder
df_test=pd.read_csv("dataset/test.csv")

In [83]:
if not os.path.isdir(os.path.join("dataset","test_submit_images")):
    os.mkdir(os.path.join("dataset","test_submit_images"))
    for f in list(df_test["image_path"].unique()):
        shutil.copy(os.path.join('dataset','images',f),os.path.join('dataset','test_submit_images'))

In [84]:
# Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5x', "/ddn/gfxhome/asislam25/projects/other_misc_projects/hackerearth/smartathon/VisualPollution/yolov5/runs/train/RoadTrainModel_yx_50_2/weights/best.pt").autoshape()
model = torch.hub.load('ultralytics/yolov5', 'custom', "/home/linux/Desktop/Smartathon/Problem 1/yolov5/runs/train/RoadTrainModel_n_50/weights/best.pt")  # custom trained model

Using cache found in /home/linux/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-1-18 Python-3.8.15 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 with Max-Q Design, 7966MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1774048 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


In [85]:
# model.load_state_dict(torch.load("/ddn/gfxhome/asislam25/projects/other_misc_projects/hackerearth/smartathon/VisualPollution/yolov5/runs/train/RoadTrainModel_yx_50_2/weights/best.pt"))

In [86]:
win_size=100

In [87]:
# Images


test_files=os.listdir("dataset/test_submit_images/")
imgs=[]
for tfile in test_files:
    imgs.append(Image.open(os.path.join("dataset/test_submit_images/",tfile)))

# img1 = Image.open('zidane.jpg')  # PIL image
# img2 = cv2.imread('bus.jpg')[:, :, ::-1]  # OpenCV image (BGR to RGB)
# imgs = [img1, img2]  # batched list of images


In [88]:
# Inference
# results = model(imgs, size=640)  # includes NMS
# win_size=100
dic_df={}
dic_df["class"]=[]
dic_df["image_path"]=[]
dic_df["name"]=[]
dic_df["xmax"]=[]
dic_df["xmin"]=[]
dic_df["ymax"]=[]
dic_df["ymin"]=[]
dic_df["confidence"]=[]


for j in range(0,len(imgs),win_size):
    print(j,win_size,len(imgs))
    imgs_subset=imgs[j:j+win_size]
    img_names_subset=test_files[j:j+win_size]
    results = model(imgs_subset)  # includes NMS
    print("size of results",len(results))
    
#     print(results)
#     break

    
    for i in range(len(results.pandas().xyxy)):
        img_name=img_names_subset[i]
        print(img_name)
        df=results.pandas().xyxy[i]
        if df.shape[0]==0:
            dic_df["class"].append("")
            dic_df["image_path"].append(img_name)
            dic_df["name"].append("")
            dic_df["xmax"].append("")
            dic_df["xmin"].append("")
            dic_df["ymax"].append("")
            dic_df["ymin"].append("")
            dic_df["confidence"].append("")
        else:
            print(img_name,"\n")
            for index,row in df.iterrows():
                xmin=int(row["xmin"])//2
                ymin=int(row["ymin"])//2
                xmax=int(row["xmax"])//2
                ymax=int(row["ymax"])//2
                class_=row["class"]
                name=row["name"]
                class_name=mapping_dic_class_name[float(name)]
                dic_df["class"].append(name)
                dic_df["image_path"].append(img_name)
                dic_df["name"].append(class_name)
                dic_df["xmax"].append(xmax)
                dic_df["xmin"].append(xmin)
                dic_df["ymax"].append(ymax)
                dic_df["ymin"].append(ymin)
                dic_df["confidence"].append(row["confidence"])

0 100 2092
size of results 100
c0ca6b67104d80f72e1f58a917586e75.jpg
c0ca6b67104d80f72e1f58a917586e75.jpg 

f80a3470f971f7f5085719017cca38ba.jpg
a9f8ca5c716934810ae720b5533485b3.jpg
a9f8ca5c716934810ae720b5533485b3.jpg 

8fe5701cb854dce9558bac81990f2681.jpg
8fe5701cb854dce9558bac81990f2681.jpg 

6abe223779ee3a1889f4d520ce73ef39.jpg
6abe223779ee3a1889f4d520ce73ef39.jpg 

42735eb9b207886a4cade197d87a84fe.jpg
42735eb9b207886a4cade197d87a84fe.jpg 

4792ee432e64122f2933802ac4a05f12.jpg
4792ee432e64122f2933802ac4a05f12.jpg 

55eda032298d933144f258d2cd9b6ec5.jpg
55eda032298d933144f258d2cd9b6ec5.jpg 

d2e33b5e382342517df8f76ab4cd19e9.jpg
d2e33b5e382342517df8f76ab4cd19e9.jpg 

ff22fbd2212505b594b65615746d2729.jpg
ff22fbd2212505b594b65615746d2729.jpg 

0081b25823f23a536c9f1952491db636.jpg
0081b25823f23a536c9f1952491db636.jpg 

9c49c884f5ad228deebe93338f57eb18.jpg
9c49c884f5ad228deebe93338f57eb18.jpg 

39acda862f5037830953db83eb0669f3.jpg
39acda862f5037830953db83eb0669f3.jpg 

e4790236df495281fe7a

a2bbf3e8c117a05f1fa20f47ea226934.jpg 

6744014e2a93b246f1e7c711708feeb4.jpg
6744014e2a93b246f1e7c711708feeb4.jpg 

609760ab21b3bf9c19e1bfc2c6aa21bf.jpg
609760ab21b3bf9c19e1bfc2c6aa21bf.jpg 

7d06df36ce2ad637eb09ca6684190550.jpg
e91eb51fcd2cea830156ee8854e5896d.jpg
e91eb51fcd2cea830156ee8854e5896d.jpg 

9fad84a12fcea37a03e9b29a9bf415e7.jpg
9fad84a12fcea37a03e9b29a9bf415e7.jpg 

ae34d0f47bc8beaa9affbc72ab4c8ea7.jpg
ae34d0f47bc8beaa9affbc72ab4c8ea7.jpg 

cf91e1f58cff6cba07aafa147802532f.jpg
cf91e1f58cff6cba07aafa147802532f.jpg 

a59353d45eb22f89e0c73aaae706dd3b.jpg
a59353d45eb22f89e0c73aaae706dd3b.jpg 

dcee0cf9016e571218ed42fcd3f8bf96.jpg
dcee0cf9016e571218ed42fcd3f8bf96.jpg 

eaf124c6e5124e9e312fc6c7d3df02d9.jpg
eaf124c6e5124e9e312fc6c7d3df02d9.jpg 

3836a1a006850aa1b8f74dde20d395a1.jpg
3836a1a006850aa1b8f74dde20d395a1.jpg 

c2ddd698752bf09938afcbc57a29e50e.jpg
ae9158b3501e7e7aed60ec06d7703802.jpg
ae9158b3501e7e7aed60ec06d7703802.jpg 

e6cafa306a42ab82f35b6c31b96ea52e.jpg
e6cafa306a42ab

dcb96f2b5bdb7fc476ccf61d510a84a8.jpg 

f69bd08699ac36e8abdb16d54124189d.jpg
f69bd08699ac36e8abdb16d54124189d.jpg 

3d46e39f820eb580d37f36f8313f75b7.jpg
5b86d80194dcca2200c066f1a26ecb43.jpg
5b86d80194dcca2200c066f1a26ecb43.jpg 

a97decd805125da66be93c9fa9798c11.jpg
a97decd805125da66be93c9fa9798c11.jpg 

439c12d525f8929cc1f1edf49d4e0aaa.jpg
439c12d525f8929cc1f1edf49d4e0aaa.jpg 

70c91870e32ec207a6336afc062cb08f.jpg
70c91870e32ec207a6336afc062cb08f.jpg 

6709bf8ddcfd126c7d553f11dda7e9bc.jpg
6709bf8ddcfd126c7d553f11dda7e9bc.jpg 

66af6c008662158eb2667a9e372125ac.jpg
66af6c008662158eb2667a9e372125ac.jpg 

3e8329a5ae51069ef4da04f2ba9f2ed5.jpg
3e8329a5ae51069ef4da04f2ba9f2ed5.jpg 

89ac50a6e1bd88a78002eebc49522821.jpg
89ac50a6e1bd88a78002eebc49522821.jpg 

33fec99cca597d69c110cbb216e35ef9.jpg
33fec99cca597d69c110cbb216e35ef9.jpg 

581bf36febbbd19a9f2d8088a5b20976.jpg
581bf36febbbd19a9f2d8088a5b20976.jpg 

9c0b0189f883b3bc4253465418ac61ed.jpg
9c0b0189f883b3bc4253465418ac61ed.jpg 

a4022e05023a

d4ce6a7c22642d91b55de360125c3736.jpg 

2806e5ba5a7aa0997b2acbd62db34cd6.jpg
2806e5ba5a7aa0997b2acbd62db34cd6.jpg 

1e1d6fae1a540611799dcc00a0686a55.jpg
1e1d6fae1a540611799dcc00a0686a55.jpg 

174d50f4b687afbec89c94e15404a307.jpg
174d50f4b687afbec89c94e15404a307.jpg 

45e299772aa7550f761faf48e3d80b80.jpg
45e299772aa7550f761faf48e3d80b80.jpg 

4ce1919c0279714e2c7fb14fe040d374.jpg
157b2ffb5e86fd64686e3683c99262e9.jpg
157b2ffb5e86fd64686e3683c99262e9.jpg 

05340fbca3ae9816f56a950e8985519f.jpg
05340fbca3ae9816f56a950e8985519f.jpg 

ffd8089216c7915c927a7203ece10f8d.jpg
ffd8089216c7915c927a7203ece10f8d.jpg 

861354ba67b904d3b1f24870e5408d53.jpg
861354ba67b904d3b1f24870e5408d53.jpg 

f07029ccca20f9924c3dc51e55fb57e5.jpg
f07029ccca20f9924c3dc51e55fb57e5.jpg 

6a27cb5a15a264d70f16d0a3f1464780.jpg
f478d4cc365c95a2fd8ea80047170f15.jpg
f478d4cc365c95a2fd8ea80047170f15.jpg 

770a4ff8dbb28a6376daa753719968da.jpg
770a4ff8dbb28a6376daa753719968da.jpg 

3887cf8d39e67c94041c9969d8dd5477.jpg
3887cf8d39e67c

f96272c164d233e5ca0b371b854158e6.jpg 

b04df000ed7bbd3f2536f310a63477b7.jpg
b04df000ed7bbd3f2536f310a63477b7.jpg 

4c0d782d4dac6e449c22d03cf5da8b35.jpg
4c0d782d4dac6e449c22d03cf5da8b35.jpg 

8a0a1f781fc46c5001cbe7523c1aa5ad.jpg
8a0a1f781fc46c5001cbe7523c1aa5ad.jpg 

29bb1d97f6adc54bc0297d72f2c8b6ef.jpg
29bb1d97f6adc54bc0297d72f2c8b6ef.jpg 

a5701108d046ef8819af0d65a4ad38bb.jpg
a5701108d046ef8819af0d65a4ad38bb.jpg 

cd6ad05e61ee246342826bf2c735029f.jpg
cd6ad05e61ee246342826bf2c735029f.jpg 

40c5f1d019b1310035c8612267e5f6d8.jpg
40c5f1d019b1310035c8612267e5f6d8.jpg 

7b4e338911777cd6461340b43bde53b4.jpg
7b4e338911777cd6461340b43bde53b4.jpg 

07c43fbcec360c0629566c213990982b.jpg
07c43fbcec360c0629566c213990982b.jpg 

013a2601f72a8aeb8d4bdbd863885108.jpg
013a2601f72a8aeb8d4bdbd863885108.jpg 

420129f069a3fd8c1792bc3fe144771b.jpg
420129f069a3fd8c1792bc3fe144771b.jpg 

3ef4e9c9e8ebdd22f21e8a7f972b84cf.jpg
3ef4e9c9e8ebdd22f21e8a7f972b84cf.jpg 

6dc09e56295226ec4d00b1d3bf69db86.jpg
6dc09e562952

...

d6b53720e130af4da44076a650937619.jpg 

6d24826f75cb435e2c58d6ea49d545fd.jpg
6d24826f75cb435e2c58d6ea49d545fd.jpg 

8c1b862a8e4b966b3d9f6cd39cc391d5.jpg
8c1b862a8e4b966b3d9f6cd39cc391d5.jpg 

afa16b8584f23b5875d0b74a5cbd3fa4.jpg
afa16b8584f23b5875d0b74a5cbd3fa4.jpg 

89b767ea95fb4277660c6e354e7ddd85.jpg
89b767ea95fb4277660c6e354e7ddd85.jpg 

88988d04fb7c6475e176d7cf315e5cc9.jpg
88988d04fb7c6475e176d7cf315e5cc9.jpg 

189be34daa59c77315015b6fe6c1647f.jpg
189be34daa59c77315015b6fe6c1647f.jpg 

7a4eeea135f0b77eace9cd640bc94a32.jpg
7a4eeea135f0b77eace9cd640bc94a32.jpg 

495af014c137557acd195960f3272f33.jpg
495af014c137557acd195960f3272f33.jpg 

dc8ef3dbc76827414a29bb308d2da1e5.jpg
dc8ef3dbc76827414a29bb308d2da1e5.jpg 

f0bd46e8938b7389363fdde0f3bbf323.jpg
f0bd46e8938b7389363fdde0f3bbf323.jpg 

67b2a5c39e4402e58e4c1ca4d2537be3.jpg
67b2a5c39e4402e58e4c1ca4d2537be3.jpg 

bd31f77e3d4e34576adbf63bcda21786.jpg
bd31f77e3d4e34576adbf63bcda21786.jpg 

4932d0b9cf95b54e3170f6568eb1a897.jpg
e41cf09fa1de

afe660165a10f3a1bbfa9b1abe445487.jpg 

0f89b662bdeb74f42b70c6a4e28f62e4.jpg
0f89b662bdeb74f42b70c6a4e28f62e4.jpg 

eaabaad89f5cfac70eecf13addd4eff7.jpg
eaabaad89f5cfac70eecf13addd4eff7.jpg 

32131fc4fad909e3db3f7a785e5d84b9.jpg
32131fc4fad909e3db3f7a785e5d84b9.jpg 

5ca8b988621b5e104d2f11568852411b.jpg
5ca8b988621b5e104d2f11568852411b.jpg 

9e9c410558f9771927072e3df9c97cb9.jpg
9e9c410558f9771927072e3df9c97cb9.jpg 

1cc45c417b8f67f7a14287c6b64cb73a.jpg
1cc45c417b8f67f7a14287c6b64cb73a.jpg 

95c1f10606658a7586d509e90819bcf2.jpg
95c1f10606658a7586d509e90819bcf2.jpg 

15180189864f64fee5d039a7d7a98403.jpg
15180189864f64fee5d039a7d7a98403.jpg 

04308179e509e34ab757f8458af858ab.jpg
de0a4746b8b37979b79587bc112adb5a.jpg
de0a4746b8b37979b79587bc112adb5a.jpg 

1764c097eee5e13169b53eb00b886199.jpg
1764c097eee5e13169b53eb00b886199.jpg 

faf0cbb28d699ce2cbe86354db46a2c4.jpg
faf0cbb28d699ce2cbe86354db46a2c4.jpg 

c998fee13a50cc61d9298f45ce2541fd.jpg
c998fee13a50cc61d9298f45ce2541fd.jpg 

e6ce43043a22

0dacd86b01d87b56e56415cc51a182ed.jpg 

a76f4823e9b96ea01c8df6103484c39c.jpg
a76f4823e9b96ea01c8df6103484c39c.jpg 

2f96e1f17c34ef8fe7b00d2527770451.jpg
6c70c3fcf7c64063d3fd160ccaf5bb09.jpg
e34ed775505802b53156855a07f8117f.jpg
e34ed775505802b53156855a07f8117f.jpg 

11898656980019d9157d92e72e0d658e.jpg
11898656980019d9157d92e72e0d658e.jpg 

52fa4be6eb54a204e7b4747c18153e67.jpg
52fa4be6eb54a204e7b4747c18153e67.jpg 

44171c30e32cfceb00522bf102f6e288.jpg
44171c30e32cfceb00522bf102f6e288.jpg 

e99220c7c524150dd03cffcff9b6456d.jpg
e99220c7c524150dd03cffcff9b6456d.jpg 

f511541324bda48b2bb86d3418216489.jpg
e43e3f2a78d54b579c2f4b93c7c6941a.jpg
e43e3f2a78d54b579c2f4b93c7c6941a.jpg 

acc690b2a158de207acc22cf8d3dd109.jpg
acc690b2a158de207acc22cf8d3dd109.jpg 

386fbef0ddd206c8e2955c16c9b36195.jpg
386fbef0ddd206c8e2955c16c9b36195.jpg 

97e0a3df92b02141566b935b68e02df0.jpg
97e0a3df92b02141566b935b68e02df0.jpg 

302de87440527521d5585f5ba25eb9c2.jpg
302de87440527521d5585f5ba25eb9c2.jpg 

95079e15ff2d19

8f5131b0d25935a7cdc277f46e705d40.jpg 

ed958123c1f03a40a7b6232fecbf4509.jpg
ed958123c1f03a40a7b6232fecbf4509.jpg 

fccd79371724e5a9aa63e2e7888dfe65.jpg
03eab36eff5829fe7c8fcc12ef0db6fe.jpg
03eab36eff5829fe7c8fcc12ef0db6fe.jpg 

832416faf8de588857681b138874c22e.jpg
832416faf8de588857681b138874c22e.jpg 

0b7b1672c6bed5a7e2f1a5618787c6f0.jpg
8af1a95b05bff00598d7eff3c94b3a13.jpg
8b29f4a25e4ab87f3e1ee68b1c2ef7b2.jpg
8b29f4a25e4ab87f3e1ee68b1c2ef7b2.jpg 

56f904f8056ee4d72a5f6bbe03ea762e.jpg
56f904f8056ee4d72a5f6bbe03ea762e.jpg 

0a490958aac3c79bb815a0f1f6825284.jpg
0a490958aac3c79bb815a0f1f6825284.jpg 

138b1dc82005b4c33e4886260649d313.jpg
138b1dc82005b4c33e4886260649d313.jpg 

581b397de6b2b1f81f5d47cfa2dcf20f.jpg
65f2672e3137876abacb94ed14663848.jpg
65f2672e3137876abacb94ed14663848.jpg 

28eac7b5c7683c3543fd48be1da77585.jpg
28eac7b5c7683c3543fd48be1da77585.jpg 

8db2ef89edad8b58403946ca2d2ea559.jpg
8db2ef89edad8b58403946ca2d2ea559.jpg 

db77fc656c27beac3f2a73f96d8e8e37.jpg
db77fc656c27beac

904c85d89f17a77b317b4e0cb675d8ac.jpg
904c85d89f17a77b317b4e0cb675d8ac.jpg 

77c01de98601411204637d6ca66fd968.jpg
77c01de98601411204637d6ca66fd968.jpg 

aec3b6cd2483fb1ccb1a72a59c08ff05.jpg
aec3b6cd2483fb1ccb1a72a59c08ff05.jpg 

ad1e205bd745489f038b1bea375d04e9.jpg
ad1e205bd745489f038b1bea375d04e9.jpg 

534a2bfb5597b75f64b097d3a2d8a9f4.jpg
534a2bfb5597b75f64b097d3a2d8a9f4.jpg 

b5d1a4e8c521bf8124e1e2432d3617e5.jpg
b5d1a4e8c521bf8124e1e2432d3617e5.jpg 

34c1bdb42324022f8e2367de2bbf96ea.jpg
8cf0b6b57dd9e1db771d49e8b8acb002.jpg
8cf0b6b57dd9e1db771d49e8b8acb002.jpg 

7b0b0d86ef4f1f72aef9026e7891149b.jpg
7b0b0d86ef4f1f72aef9026e7891149b.jpg 

f83ee337e61b9b1f382697339241e368.jpg
f83ee337e61b9b1f382697339241e368.jpg 

8213aba8ee8c7bbaae2b30b1cf6847af.jpg
8213aba8ee8c7bbaae2b30b1cf6847af.jpg 

a837318d2dee23ee441c9cf7687bc649.jpg
a837318d2dee23ee441c9cf7687bc649.jpg 

38ea31b13df5afed86b96c658744422b.jpg
38ea31b13df5afed86b96c658744422b.jpg 

fa13a3a6a7c218664b3692c5b7f05d48.jpg
569dbd3553b269

In [89]:
dic_df_res=pd.DataFrame(dic_df)


# In[11]:


dic_df_res.shape


# In[15]:


(5529, 8)

In [90]:
dic_df_res.head()

,class,image_path,name,xmax,xmin,ymax,ymin,confidence
0,2.0,c0ca6b67104d80f72e1f58a917586e75.jpg,POTHOLES,330,168,454,292,0.473808
1,2.0,c0ca6b67104d80f72e1f58a917586e75.jpg,POTHOLES,366,230,401,263,0.431924
2,2.0,c0ca6b67104d80f72e1f58a917586e75.jpg,POTHOLES,672,513,497,282,0.394484
3,2.0,c0ca6b67104d80f72e1f58a917586e75.jpg,POTHOLES,393,269,367,252,0.393557
4,2.0,c0ca6b67104d80f72e1f58a917586e75.jpg,POTHOLES,298,107,509,321,0.325818


In [91]:
dic_df_res=dic_df_res.drop(columns=["confidence"])

In [92]:


dic_df_res.to_csv("dataset/test_submission(n_50).csv",index=False)
